In [1]:
import pandas as pd
import numpy as np
import keras
from build_model import build_model

data_x = pd.read_csv('./data/weather/v3/2013/merged/merged_2013_h0.csv', index_col = 'Time')
data_x.columns

x = data_x[['temperature-460', 'cloudiness-460',
            'temperature-481', 'cloudiness-481',
            'temperature-645', 'cloudiness-645',]]

data_y = pd.read_csv('./data/energy/v5/2013/Consumption_2013.csv', index_col = 'Time')
data = x.copy()
data['cons'] = data_y['Auvergne-Rhone-Alpes']


data.drop(data.index[0], inplace = True)

print(data.isnull().values.any())

print(data.isnull().sum())

Using TensorFlow backend.


False
temperature-460    0
cloudiness-460     0
temperature-481    0
cloudiness-481     0
temperature-645    0
cloudiness-645     0
cons               0
dtype: int64


In [2]:
data_train = data.values
x_train = data_train[:, 0:6]
y_train = data_train[:, 6]

In [3]:
x_train[:5, :]

array([[8.4, 8, 9.5, 5, 7.1, 7],
       [8.3, 8, 9.6, 7, 7.4, 8],
       [8.6, 8, 9.5, 7, 7.6, 6],
       [8.5, 8, 9.5, 8, 8.0, 7],
       [8.5, 7, 9.4, 8, 7.9, 7]], dtype=object)

In [4]:
y_train[:5]

array(['7944', '7882', '7709', '7383', '7158'], dtype=object)

In [ ]:
#data.isnull().values.any()
#data.isnull().sum()

In [5]:
# Before
x_min = np.min(x_train)
print(x_min)

x_max = np.max(x_train)
print(x_max)

x_train = (x_train - np.min(x_train)) / (np.max(x_train) - np.min(x_train))

# After 
x_min = np.min(x_train)
print(x_min)

x_max = np.max(x_train)
print(x_max)

-7.7
36.1
0.0
1.0


In [ ]:
'''y_train = (y_train - np.min(y_train)) / (np.max(y_train) - np.min(y_train))'''

# Test data

In [6]:
data_x = pd.read_csv('./data/weather/v3/2014/merged/merged_2014_h0.csv', index_col = 'Time')
data_x.columns

x = data_x[['temperature-460', 'cloudiness-460',
            'temperature-481', 'cloudiness-481',
            'temperature-645', 'cloudiness-645',]]

data_y = pd.read_csv('./data/energy/v5/2014/Consumption_2014.csv', index_col = 'Time')
data = x.copy()
data['cons'] = data_y['Auvergne-Rhone-Alpes']


#data.drop(data.index[0], inplace = True)

print(data.isnull().values.any())

print(data.isnull().sum())

data_test = data.values
x_test = data_test[:, 0:6]
y_test = data_test[:, 6]

False
temperature-460    0
cloudiness-460     0
temperature-481    0
cloudiness-481     0
temperature-645    0
cloudiness-645     0
cons               0
dtype: int64


In [7]:
seq_len = 100
def data_to_sequence(data, n_prev = seq_len):  
    
    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data[i:i+n_prev, 0:6])
        docY.append(data[i+n_prev, 6])
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

In [8]:
x_train, y_train = data_to_sequence(data_train)

In [9]:
x_test, y_test = data_to_sequence(data_test)

In [10]:
model = build_model(features = 6, seq_len = seq_len, out = 1)
model.summary()

from keras.callbacks import TensorBoard , ModelCheckpoint, ReduceLROnPlateau


tbCallBack = TensorBoard(log_dir ='./logs/', 
                         histogram_freq = 0, 
                         write_graph = True)

filepath = "best_model.hdf5"
best_model = ModelCheckpoint(filepath = filepath, 
                             monitor = 'val_loss', 
                             verbose = 1, 
                             save_best_only = True, 
                             save_weights_only = False, 
                             mode = 'auto', period = 1)


reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2)

build_model.py:16: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(50, 10, padding="valid")`
  model.add(Convolution1D(50, 10, border_mode='valid'))


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
('> Compilation Time : ', 0.02442312240600586)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 100)          42800     
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 91, 50)            50050     
_________________________________________________________________
activation_2 (Activation)    (None, 91, 50)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4550)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4551      
____________

In [11]:
model.fit(x_train,
          y_train,
          batch_size = 100,
          epochs = 100,
          validation_data = [x_test, y_test],
          callbacks = [best_model, tbCallBack, reduce_lr])

Train on 8657 samples, validate on 8633 samples
Epoch 1/100
8657/8657 [==============================] - 17s 2ms/step - loss: 19.8672 - val_loss: 9.8549

Epoch 00001: val_loss improved from inf to 9.85494, saving model to best_model.hdf5
Epoch 2/100
8657/8657 [==============================] - 18s 2ms/step - loss: 10.0189 - val_loss: 23.5359

Epoch 00002: val_loss did not improve from 9.85494
Epoch 3/100
8657/8657 [==============================] - 19s 2ms/step - loss: 9.9458 - val_loss: 10.2797

Epoch 00003: val_loss did not improve from 9.85494
Epoch 4/100
8657/8657 [==============================] - 20s 2ms/step - loss: 8.7608 - val_loss: 9.3252

Epoch 00004: val_loss improved from 9.85494 to 9.32521, saving model to best_model.hdf5
Epoch 5/100
8657/8657 [==============================] - 20s 2ms/step - loss: 8.1641 - val_loss: 9.6551

Epoch 00005: val_loss did not improve from 9.32521
Epoch 6/100
8657/8657 [==============================] - 21s 2ms/step - loss: 8.4973 - val_loss: 9


Epoch 00051: val_loss did not improve from 9.07947
Epoch 52/100
8657/8657 [==============================] - 25s 3ms/step - loss: 6.8399 - val_loss: 9.1390

Epoch 00052: val_loss did not improve from 9.07947
Epoch 53/100
8657/8657 [==============================] - 28s 3ms/step - loss: 6.8399 - val_loss: 9.1390

Epoch 00053: val_loss did not improve from 9.07947
Epoch 54/100
8657/8657 [==============================] - 26s 3ms/step - loss: 6.8399 - val_loss: 9.1390

Epoch 00054: val_loss did not improve from 9.07947
Epoch 55/100
8657/8657 [==============================] - 24s 3ms/step - loss: 6.8399 - val_loss: 9.1390

Epoch 00055: val_loss did not improve from 9.07947
Epoch 56/100
8657/8657 [==============================] - 26s 3ms/step - loss: 6.8399 - val_loss: 9.1390

Epoch 00056: val_loss did not improve from 9.07947
Epoch 57/100
8657/8657 [==============================] - 23s 3ms/step - loss: 6.8399 - val_loss: 9.1390

Epoch 00057: val_loss did not improve from 9.07947
Epoch 